In [1]:
using Qaintessent
using Qaintellect
using Flux
using LinearAlgebra
using IterTools: ncycle
using Distributions
using StatsBase

In [2]:
#Quantum Generator Circuit
n = 4
k = 2
theta = sqrt(2)
cgs = [
    circuit_gate(4, RyGate(theta)),
    circuit_gate(3, RyGate(theta)),
    circuit_gate(2, RyGate(theta)),
    circuit_gate(1, RyGate(theta)),
    
    circuit_gate(3, ZGate(), 4),
    circuit_gate(2, ZGate(), 3),
    circuit_gate(1, ZGate(), 2),
    circuit_gate(4, ZGate(), 1),
    
    circuit_gate(4, RyGate(theta)),
    circuit_gate(3, RyGate(theta)),
    circuit_gate(2, RyGate(theta)),
    circuit_gate(1, RyGate(theta)),
    
    circuit_gate(3, ZGate(), 4),
    circuit_gate(2, ZGate(), 3),
    circuit_gate(1, ZGate(), 2),
    circuit_gate(4, ZGate(), 1),
    
    circuit_gate(4, RyGate(theta)),
    circuit_gate(3, RyGate(theta)),
    circuit_gate(2, RyGate(theta)),
    circuit_gate(1, RyGate(theta)),
]

circ = Circuit{4}(cgs)


    4 —[Ry]———•————————————————[Z ]——[Ry]———•————————————————[Z ]——[Ry]—
              |                 |           |                 |         
    3 —[Ry]——[Z ]———•————————————————[Ry]——[Z ]———•————————————————[Ry]—
                    |           |                 |           |         
    2 —[Ry]————————[Z ]———•——————————[Ry]————————[Z ]———•——————————[Ry]—
                          |     |                       |     |         
    1 —[Ry]——————————————[Z ]———•————[Ry]——————————————[Z ]———•————[Ry]—


In [3]:
#Bitstrings representing states
d = Normal(1.5,1.5)
d = truncated(d, 0, 3)
s = Array{String}(undef, 10)
for m = 0:9
    num = rand(d)
    num = floor(Int, round(num))
    a = bitstring(num)
    s[m+1] = a[end-n+1:end]
end
print(s)

["0010", "0001", "0010", "0001", "0010", "0000", "0010", "0001", "0010", "0010"]

In [4]:
#Real state to be replicated
ψreal = rand(d, 16)
ψreal /= norm(ψreal)
preal = abs2.(ψreal)

16-element Vector{Float64}:
 0.19914370214867827
 0.003880964255501065
 0.016931633396368634
 0.004314646368910078
 0.1681907528366119
 0.0734072607891125
 0.00019631652835125962
 0.012051344421094762
 0.14992027902902524
 0.008311829975208095
 0.05026213843368382
 0.0077622320444383835
 0.06534352187711481
 0.11135058812445461
 0.012242458112846617
 0.11669033165859995

In [5]:
#Classical Discriminator
disc = Chain(
      Dense(4, 20, leakyrelu),
      Dense(20, 20, leakyrelu),
      Dense(20, 1, sigmoid)
    )

Chain(Dense(4, 20, leakyrelu), Dense(20, 20, leakyrelu), Dense(20, 1, σ))

In [6]:
#Input to the generator
ψ = rand(d, 16)
ψ /= norm(ψ)
ψ = convert(Vector{ComplexF64}, ψ)

16-element Vector{ComplexF64}:
   0.0678341714345371 + 0.0im
   0.3996293069688195 + 0.0im
  0.29027419544652966 + 0.0im
  0.19758968626223708 + 0.0im
  0.24757480591863165 + 0.0im
  0.36215766077337697 + 0.0im
  0.20038485531453545 + 0.0im
   0.3907557307441562 + 0.0im
  0.15489020518196683 + 0.0im
  0.13315943890401405 + 0.0im
 0.060523294923083906 + 0.0im
  0.21304785542097168 + 0.0im
  0.07920392873467816 + 0.0im
  0.36492930702014825 + 0.0im
  0.12735231152826876 + 0.0im
  0.28400426043913396 + 0.0im

In [7]:
#discriminator action matrices
function createD_actn(disc)
    d_actn = [log(disc([0;0;0;0])[1]) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
          0 log(disc([0;0;0;1])[1]) 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
          0 0 log(disc([0;0;1;0])[1]) 0 0 0 0 0 0 0 0 0 0 0 0 0;
          0 0 0 log(disc([0;0;1;1])[1]) 0 0 0 0 0 0 0 0 0 0 0 0;
          0 0 0 0 log(disc([0;1;0;0])[1]) 0 0 0 0 0 0 0 0 0 0 0;
          0 0 0 0 0 log(disc([0;1;0;1])[1]) 0 0 0 0 0 0 0 0 0 0;
          0 0 0 0 0 0 log(disc([0;1;1;0])[1]) 0 0 0 0 0 0 0 0 0;
          0 0 0 0 0 0 0 log(disc([0;1;1;1])[1]) 0 0 0 0 0 0 0 0;
          0 0 0 0 0 0 0 0 log(disc([1;0;0;0])[1]) 0 0 0 0 0 0 0;
          0 0 0 0 0 0 0 0 0 log(disc([1;0;0;1])[1]) 0 0 0 0 0 0;
          0 0 0 0 0 0 0 0 0 0 log(disc([1;0;1;0])[1]) 0 0 0 0 0;
          0 0 0 0 0 0 0 0 0 0 0 log(disc([1;0;1;1])[1]) 0 0 0 0;
          0 0 0 0 0 0 0 0 0 0 0 0 log(disc([1;1;0;0])[1]) 0 0 0;
          0 0 0 0 0 0 0 0 0 0 0 0 0 log(disc([1;1;0;1])[1]) 0 0;
          0 0 0 0 0 0 0 0 0 0 0 0 0 0 log(disc([1;1;1;0])[1]) 0;
          0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 log(disc([1;1;1;1])[1])]
    return d_actn
end

function createD_actn_alt(disc)
    d_actn_alt = [log(1-disc([0;0;0;0])[1]) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
          0 log(1-disc([0;0;0;1])[1]) 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
          0 0 log(1-disc([0;0;1;0])[1]) 0 0 0 0 0 0 0 0 0 0 0 0 0;
          0 0 0 log(1-disc([0;0;1;1])[1]) 0 0 0 0 0 0 0 0 0 0 0 0;
          0 0 0 0 log(1-disc([0;1;0;0])[1]) 0 0 0 0 0 0 0 0 0 0 0;
          0 0 0 0 0 log(1-disc([0;1;0;1])[1]) 0 0 0 0 0 0 0 0 0 0;
          0 0 0 0 0 0 log(1-disc([0;1;1;0])[1]) 0 0 0 0 0 0 0 0 0;
          0 0 0 0 0 0 0 log(1-disc([0;1;1;1])[1]) 0 0 0 0 0 0 0 0;
          0 0 0 0 0 0 0 0 log(1-disc([1;0;0;0])[1]) 0 0 0 0 0 0 0;
          0 0 0 0 0 0 0 0 0 log(1-disc([1;0;0;1])[1]) 0 0 0 0 0 0;
          0 0 0 0 0 0 0 0 0 0 log(1-disc([1;0;1;0])[1]) 0 0 0 0 0;
          0 0 0 0 0 0 0 0 0 0 0 log(1-disc([1;0;1;1])[1]) 0 0 0 0;
          0 0 0 0 0 0 0 0 0 0 0 0 log(1-disc([1;1;0;0])[1]) 0 0 0;
          0 0 0 0 0 0 0 0 0 0 0 0 0 log(1-disc([1;1;0;1])[1]) 0 0;
          0 0 0 0 0 0 0 0 0 0 0 0 0 0 log(1-disc([1;1;1;0])[1]) 0;
          0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 log(1-disc([1;1;1;1])[1])]
    return d_actn_alt
end

createD_actn_alt (generic function with 1 method)

In [8]:
function genLoss(initial)
    res = apply(initial, circ.moments)
    fake = abs2.(res)
    d_actn = createD_actn(disc)
    gen_loss = sum(diag(fake .* -d_actn))
    return gen_loss
end

genLoss (generic function with 1 method)

In [9]:
function discLoss(initial, real)
    res = apply(initial, circ.moments)
    fake = abs2.(res)
    d_actn = createD_actn(disc)
    d_actn_alt = createD_actn_alt(disc)
    disc_loss = sum(diag(real .* d_actn + fake .* d_actn_alt))
    return disc_loss
end

discLoss (generic function with 1 method)

In [10]:
#Training Discriminator
function trainDiscriminator()
    disc_data = ncycle([(ψ, preal)], 1)
    disc_opt = ADAM(0.00001, (0.9, 0.85))
    disc_params = Flux.params(disc)
    evalcb() = @show(discLoss(ψ, preal))
    Flux.Optimise.train!(discLoss, disc_params, disc_data, disc_opt, cb=Flux.throttle(evalcb, 0.01))
end    

trainDiscriminator (generic function with 1 method)

In [11]:
#Training Generator
function trainGenerator()
    gen_data = ncycle([(ψ)], 1)
    gen_opt = ADAM(0.00001, (0.9, 0.85))
    gen_params = Flux.params(circ)
    evalcb() = @show(genLoss(ψ))
    Flux.Optimise.train!(genLoss, gen_params, gen_data, gen_opt, cb=Flux.throttle(evalcb, 0.01))
end

trainGenerator (generic function with 1 method)

In [12]:
epochs = 400
for e in 1:epochs
    trainDiscriminator()
    trainGenerator()
end

discLoss(ψ, preal) = -1.4185880524354588
genLoss(ψ) = 0.541876298587422
discLoss(ψ, preal) = -1.4186590121359333
genLoss(ψ) = 0.5417978774355757
discLoss(ψ, preal) = -1.4187299946889032
genLoss(ψ) = 0.5417194503081315
discLoss(ψ, preal) = -1.4188010540622822
genLoss(ψ) = 0.5416409037528798
discLoss(ψ, preal) = -1.4188721346294535
genLoss(ψ) = 0.5415623145463988
discLoss(ψ, preal) = -1.418943194704779
genLoss(ψ) = 0.5414837895997658
discLoss(ψ, preal) = -1.4190143282201828
genLoss(ψ) = 0.5414052445057257
discLoss(ψ, preal) = -1.4190854627792313
genLoss(ψ) = 0.5413267895817677
discLoss(ψ, preal) = -1.419156476072363
genLoss(ψ) = 0.5412483942752959
discLoss(ψ, preal) = -1.4192273574774734
genLoss(ψ) = 0.5411701325726864
discLoss(ψ, preal) = -1.4192983257689553
genLoss(ψ) = 0.5410919011702747
discLoss(ψ, preal) = -1.4193693243253307
genLoss(ψ) = 0.5410135860255122
discLoss(ψ, preal) = -1.419440420072759
genLoss(ψ) = 0.5409353163944075
discLoss(ψ, preal) = -1.419511415376557
genLoss(ψ) = 0.

discLoss(ψ, preal) = -1.427251025178403
genLoss(ψ) = 0.5325399875594842
discLoss(ψ, preal) = -1.4273245021865841
genLoss(ψ) = 0.5324627861201344
discLoss(ψ, preal) = -1.4273979800083172
genLoss(ψ) = 0.532385510943554
discLoss(ψ, preal) = -1.4274714831585484
genLoss(ψ) = 0.532308269745515
discLoss(ψ, preal) = -1.427545036407964
genLoss(ψ) = 0.5322310573977405
discLoss(ψ, preal) = -1.4276185589888875
genLoss(ψ) = 0.5321538245719595
discLoss(ψ, preal) = -1.4276922072367286
genLoss(ψ) = 0.5320765963829076
discLoss(ψ, preal) = -1.4277658222928
genLoss(ψ) = 0.5319993917684772
discLoss(ψ, preal) = -1.4278394522781506
genLoss(ψ) = 0.5319221735594217
discLoss(ψ, preal) = -1.427913121276267
genLoss(ψ) = 0.5318450234524907
discLoss(ψ, preal) = -1.427986781040698
genLoss(ψ) = 0.5317678771913117
discLoss(ψ, preal) = -1.4280605266857611
genLoss(ψ) = 0.5316907423079572
discLoss(ψ, preal) = -1.4281343108672213
genLoss(ψ) = 0.5316135069694574
discLoss(ψ, preal) = -1.428208062389292
genLoss(ψ) = 0.53153

genLoss(ψ) = 0.523886216182955
discLoss(ψ, preal) = -1.4357155783160624
genLoss(ψ) = 0.5238129727488189
discLoss(ψ, preal) = -1.435787495164983
genLoss(ψ) = 0.5237395942635399
discLoss(ψ, preal) = -1.4358593270063695
genLoss(ψ) = 0.5236663276312553
discLoss(ψ, preal) = -1.4359311749308232
genLoss(ψ) = 0.523593088681929
discLoss(ψ, preal) = -1.4360030244603923
genLoss(ψ) = 0.5235198461202454
discLoss(ψ, preal) = -1.4360749973400804
genLoss(ψ) = 0.5234466115243904
discLoss(ψ, preal) = -1.4361469194879328
genLoss(ψ) = 0.5233733448572042
discLoss(ψ, preal) = -1.4362188340048032
genLoss(ψ) = 0.523300131212928
discLoss(ψ, preal) = -1.4362908109840018
genLoss(ψ) = 0.5232269347268127
discLoss(ψ, preal) = -1.436362818380592
genLoss(ψ) = 0.523153665585493
discLoss(ψ, preal) = -1.436434840279479
genLoss(ψ) = 0.5230804785640271
discLoss(ψ, preal) = -1.4365068778801326
genLoss(ψ) = 0.5230072684348261
discLoss(ψ, preal) = -1.4365789839215886
genLoss(ψ) = 0.5229340416263606
discLoss(ψ, preal) = -1.43

genLoss(ψ) = 0.5153854294265805
discLoss(ψ, preal) = -1.4442374685502306
genLoss(ψ) = 0.5153141368311612
discLoss(ψ, preal) = -1.4443101952233075
genLoss(ψ) = 0.5152428545410985
discLoss(ψ, preal) = -1.4443829543876001
genLoss(ψ) = 0.5151715949548299
discLoss(ψ, preal) = -1.4444557453157245
genLoss(ψ) = 0.5151003138264343
discLoss(ψ, preal) = -1.444528585014909
genLoss(ψ) = 0.5150290415898349
discLoss(ψ, preal) = -1.4446014432049075
genLoss(ψ) = 0.5149577928618843
discLoss(ψ, preal) = -1.4446743247268081
genLoss(ψ) = 0.5148865101465399
discLoss(ψ, preal) = -1.4447472283049807
genLoss(ψ) = 0.5148152638881156
discLoss(ψ, preal) = -1.4448201100490632
genLoss(ψ) = 0.5147440431008956
discLoss(ψ, preal) = -1.4448930859942704
genLoss(ψ) = 0.5146728194261703
discLoss(ψ, preal) = -1.4449659873547511
genLoss(ψ) = 0.514601624095793
discLoss(ψ, preal) = -1.4450389915156296
genLoss(ψ) = 0.514530419834532
discLoss(ψ, preal) = -1.4451120592598556
genLoss(ψ) = 0.5144591436835514
discLoss(ψ, preal) = -

In [13]:
real = preal
fake = apply(ψ, circ.moments)

16-element Vector{ComplexF64}:
    0.5808142835634119 + 0.0im
   0.11077009131221668 + 0.0im
   -0.2548736155341891 + 0.0im
   0.07230573722778237 + 0.0im
  0.012655429487708425 + 0.0im
    0.3186075354718224 + 0.0im
    0.0645101950361634 + 0.0im
  0.021026287529858086 + 0.0im
  -0.25195048481697296 + 0.0im
  0.032547546542407685 + 0.0im
    0.3631917370362945 + 0.0im
  0.017228435074208515 + 0.0im
 -0.032164282202353206 + 0.0im
  -0.14446048301381836 + 0.0im
  -0.12561706451624427 + 0.0im
      -0.4893822823215 + 0.0im

In [14]:
real

16-element Vector{Float64}:
 0.19914370214867827
 0.003880964255501065
 0.016931633396368634
 0.004314646368910078
 0.1681907528366119
 0.0734072607891125
 0.00019631652835125962
 0.012051344421094762
 0.14992027902902524
 0.008311829975208095
 0.05026213843368382
 0.0077622320444383835
 0.06534352187711481
 0.11135058812445461
 0.012242458112846617
 0.11669033165859995

In [15]:
err = 0
for i in 1:16
    err += abs2.(real[i]-fake[i])
end    

In [16]:
err

1.045508196911623